<a href="https://colab.research.google.com/github/abelalanoca/P.MAGNO/blob/main/Proy%20ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
from google.colab import files
from google.colab import drive
import pandas as pd

# 1️⃣ Subir archivo desde tu PC
uploaded = files.upload()

# 2️⃣ Montar Google Drive (si quieres guardar/leer desde ahí)
drive.mount("/content/drive")

# 3️⃣ Leer hoja 1 del archivo subido
nombre_archivo = list(uploaded.keys())[0]
df_sheet1 = pd.read_excel(nombre_archivo, sheet_name=0)

# 4️⃣ Ver primeras filas
df_sheet1.head()



Saving B-2201-my2025.XLS to B-2201-my2025 (8).XLS
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83
0,Balance General por Empresa Bancaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(En miles de soles),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Activo,Banco BBVA Perú,NaN,NaN,NaN,BANCOM,NaN,NaN,NaN,Banco de Crédito del Perú,...,NaN,NaN,NaN,Banco de Crédito con Sucursales en el \nExterior,NaN,NaN,NaN,Total Banca Múltiple Incluye Sucursales en el ...,NaN,NaN


In [77]:
# 3️⃣ Definir función para extraer listas
def extraer_listas_balance(df):
    # --- Cuentas del Activo ---
    idx_activo = df[df.iloc[:, 0].astype(str).str.strip().str.upper() == "ACTIVO"].index[0]
    idx_total_activo = df[df.iloc[:, 0].astype(str).str.strip().str.upper() == "TOTAL ACTIVO"].index[0]

    cuentas_del_activo = df.iloc[idx_activo:idx_total_activo + 1, 0] \
        .dropna().astype(str).str.strip().tolist()
    if cuentas_del_activo[0].upper() == "ACTIVO":
        cuentas_del_activo = cuentas_del_activo[1:]

    # --- Lista de Pasivos ---
    idx_pasivo = df[df.iloc[:, 0].astype(str).str.strip().str.upper() == "PASIVO"].index[0]
    idx_otras_conting = df[df.iloc[:, 0].astype(str).str.strip().str.upper() == "OTRAS CUENTAS CONTINGENTES"].index[0]

    lista_pasivos = df.iloc[idx_pasivo:idx_otras_conting + 1, 0] \
        .dropna().astype(str).str.strip().tolist()

    # Quitar "Pasivo" si está presente
    lista_pasivos = [item for item in lista_pasivos if item.upper() != "PASIVO"]

    # --- Bancos en la fila "Activo" ---
    bancos_activo = df.iloc[idx_activo, 1:].dropna().astype(str).str.strip().tolist()
    bancos_activo = [b for b in bancos_activo if b.upper() != "ACTIVO"]

    return cuentas_del_activo, lista_pasivos, bancos_activo

# 4️⃣ Ejecutar función y mostrar resultados
cuentas_del_activo, lista_pasivos, bancos_activo = extraer_listas_balance(df_sheet1)

print("📌 Cuentas del Activo:")
print(cuentas_del_activo)

print("\n📌 Lista de Pasivos:")
print(lista_pasivos)

print("\n📌 Bancos en Activo:")
print(bancos_activo)


📌 Cuentas del Activo:
['DISPONIBLE', 'Caja', 'Bancos y Corresponsales', 'Canje', 'Otros', 'FONDOS INTERBANCARIOS', 'INVERSIONES NETAS DE PROVISIONES', 'Inversiones a Valor Razonable con Cambios en Resultados', 'Inversiones Disponibles para la Venta', 'Inversiones a Vencimiento', 'Inversiones en Subsidiarias, Asociadas y Negocios Conjuntos', 'Inversiones en Commodities', 'Provisiones', 'CRÉDITOS NETOS DE PROVISIONES Y DE INGRESOS NO DEVENGADOS', 'Vigentes', 'Cuentas Corrientes', 'Tarjetas de Crédito', 'Descuentos', 'Factoring', 'Préstamos', 'Arrendamiento Financiero', 'Hipotecarios para Vivienda', 'Comercio Exterior', 'Créditos por Liquidar', 'Otros', 'Refinanciados y Reestructurados', 'Atrasados', 'Vencidos', 'En Cobranza Judicial', 'Provisiones', 'Intereses y Comisiones no Devengados', 'CUENTAS POR COBRAR NETAS DE PROVISIONES', 'RENDIMIENTOS POR COBRAR', 'Disponible', 'Fondos Interbancarios', 'Inversiones', 'Créditos', 'Cuentas por Cobrar', 'BIENES REALIZABLES, RECIBIDOS EN PAGO Y ADJ

In [78]:
# 2️⃣ Función para extraer fecha con formato Mes-Día (ej: Feb-31)
def extraer_fecha(df):
    # Buscar índice de la fila que contiene "Balance General por Empresa Bancaria"
    idx_balance = df[df.astype(str).apply(lambda row: row.str.contains("Balance General por Empresa Bancaria", case=False, na=False)).any(axis=1)].index[0]

    # Obtener la celda de abajo (donde está la fecha)
    fecha_cruda = df.iloc[idx_balance + 1].dropna().astype(str).tolist()[0]

    # Convertir a datetime y formatear como Mes-Día
    try:
        fecha_dt = pd.to_datetime(fecha_cruda, errors='coerce')
        if pd.notnull(fecha_dt):
            fecha_formateada = fecha_dt.strftime("%b-%d")  # Ejemplo: Feb-31
        else:
            fecha_formateada = fecha_cruda
    except:
        fecha_formateada = fecha_cruda

    return [fecha_formateada]  # Lista con un solo elemento

# 3️⃣ Ejecutar función
Fecha = extraer_fecha(df_sheet1)

# 4️⃣ Mostrar lista
print("📅 Lista Fecha:", Fecha)

📅 Lista Fecha: ['May-31']


In [79]:
import pandas as pd

def crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos):
    # Combinar todas las cuentas
    columnas_cuentas = cuentas_del_activo + lista_pasivos

    # Crear estructura vacía
    df_2 = pd.DataFrame(columns=["MES", "ENTIDAD", "MONEDA"] + columnas_cuentas)

    # Lista final de entidades repetidas 3 veces (MN, ME, TOTAL)
    entidades_repetidas = []
    monedas_repetidas = []
    meses_repetidos = []

    for banco in bancos_activo:
        for moneda in ["MN", "ME", "TOTAL"]:
            entidades_repetidas.append(banco)
            monedas_repetidas.append(moneda)
            meses_repetidos.append(Fecha[0])  # Siempre misma fecha para todos

    # Llenar las columnas fijas
    df_2["MES"] = meses_repetidos
    df_2["ENTIDAD"] = entidades_repetidas
    df_2["MONEDA"] = monedas_repetidas

    return df_2

# Crear df_2
df_2 = crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos)

In [80]:
def llenar_df_2_activo_matriz(df_origen, df_2, bancos_activo, cuentas_del_activo):
    # 1️⃣ Limpiar textos
    df_limpio = df_origen.copy()
    df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)

    # 2️⃣ Localizar índice de "ACTIVO"
    idx_activo = df_limpio[df_limpio.iloc[:, 0].str.upper() == "ACTIVO"].index[0]

    # 3️⃣ Filas con cuentas del activo (debajo de "ACTIVO")
    filas_cuentas = {}
    for cuenta in cuentas_del_activo:
        match = df_limpio.index[df_limpio.iloc[:, 0].str.upper() == cuenta.upper()].tolist()
        if match:
            filas_cuentas[cuenta] = match[0]

    # 4️⃣ Columnas con bancos (a la derecha de "ACTIVO")
    columnas_bancos = {}
    for banco in bancos_activo:
        match = df_limpio.columns[df_limpio.iloc[idx_activo, :].astype(str) == banco].tolist()
        if match:
            col_index = df_limpio.columns.get_loc(match[0])
            columnas_bancos[banco] = col_index

    # 5️⃣ Recorrer banco-cuenta y llenar valores MN, ME, TOTAL
    for banco, col_idx in columnas_bancos.items():
        for cuenta, fila_idx in filas_cuentas.items():
            valores = df_origen.iloc[fila_idx, col_idx:col_idx+3].tolist()
            for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                df_2.loc[(df_2["ENTIDAD"] == banco) & (df_2["MONEDA"] == moneda), cuenta] = valor

    return df_2


In [81]:
def llenar_df_2_pasivo_matriz(df_origen, df_2, bancos_activo, lista_pasivos):
    # 1️⃣ Limpiar textos
    df_limpio = df_origen.copy()
    df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)

    # 2️⃣ Localizar índice de "PASIVO"
    idx_pasivo = df_limpio[df_limpio.iloc[:, 0].str.upper() == "PASIVO"].index[0]

    # 3️⃣ Filas con cuentas del pasivo (debajo de "PASIVO")
    filas_cuentas = {}
    for cuenta in lista_pasivos:
        match = df_limpio.index[df_limpio.iloc[:, 0].str.upper() == cuenta.upper()].tolist()
        if match:
            filas_cuentas[cuenta] = match[0]

    # 4️⃣ Columnas con bancos (a la derecha de "PASIVO")
    columnas_bancos = {}
    for banco in bancos_activo:
        match = df_limpio.columns[df_limpio.iloc[idx_pasivo, :].astype(str) == banco].tolist()
        if match:
            col_index = df_limpio.columns.get_loc(match[0])
            columnas_bancos[banco] = col_index

    # 5️⃣ Recorrer banco-cuenta y llenar valores MN, ME, TOTAL
    for banco, col_idx in columnas_bancos.items():
        for cuenta, fila_idx in filas_cuentas.items():
            valores = df_origen.iloc[fila_idx, col_idx:col_idx+3].tolist()
            for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                df_2.loc[(df_2["ENTIDAD"] == banco) & (df_2["MONEDA"] == moneda), cuenta] = valor

    return df_2


In [82]:
# Llenar primero ACTIVO
df_2 = llenar_df_2_activo_matriz(df_sheet1, df_2, bancos_activo, cuentas_del_activo)

# Luego PASIVO
df_2 = llenar_df_2_pasivo_matriz(df_sheet1, df_2, bancos_activo, lista_pasivos)

print(df_2.head(20))


/tmp/ipython-input-3928686145.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
/tmp/ipython-input-3438186554.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)


       MES                           ENTIDAD MONEDA    DISPONIBLE  \
0   May-31                   Banco BBVA Perú     MN    2910359.14   
1   May-31                   Banco BBVA Perú     ME  17611783.971   
2   May-31                   Banco BBVA Perú  TOTAL  20522143.111   
3   May-31                            BANCOM     MN    178740.521   
4   May-31                            BANCOM     ME    126950.941   
5   May-31                            BANCOM  TOTAL    305691.463   
6   May-31         Banco de Crédito del Perú     MN    5638212.66   
7   May-31         Banco de Crédito del Perú     ME  34914541.606   
8   May-31         Banco de Crédito del Perú  TOTAL  40552754.266   
9   May-31                   Banco Pichincha     MN    478629.077   
10  May-31                   Banco Pichincha     ME    974854.705   
11  May-31                   Banco Pichincha  TOTAL   1453483.783   
12  May-31  Banco Interamericano de Finanzas     MN    350159.391   
13  May-31  Banco Interamericano d

In [89]:
# Guardar el DataFrame como archivo Excel
df_2.to_excel("mi_archivo_limpio.xlsx", index=False)

# Descargar el archivo a tu PC
from google.colab import files
files.download("mi_archivo_limpio.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>